## Imports

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
sys.path.append('../')
# This is only needed for jupyter notebook
import json
import networkx as nx
import pandas as pd
import numpy as np

## Read networks from biogrid files

In [2]:
#from gmundo.network_op import read_network_from_biogrid_file

ghuman = nx.read_edgelist("../data/biogrid_files/human.tsv")
gmouse = nx.read_edgelist("../data/biogrid_files/mouse.tsv")

human_nodelist = list(ghuman.nodes())
mouse_nodelist = list(gmouse.nodes())

#save edge lists as tsv
#nx.write_edgelist(ghuman, "human.tsv", data=False, delimiter='\t')
#nx.write_edgelist(gmouse, "mouse.tsv", data=False, delimiter='\t')

# SAVING THE JSON FILES
with open("human.json", "w") as hj:
    human_map = {k:i for i, k in enumerate(human_nodelist)}
    json.dump(human_map, hj)
with open("mouse.json", "w") as mj:
    mouse_map = {k:i for i, k in enumerate(mouse_nodelist)}
    json.dump(mouse_map, mj)



In [56]:
ghuman = nx.read_edgelist("../data/biogrid_files/mouse.tsv")
gmouse = nx.read_edgelist("../data/biogrid_files/human.tsv")
ghuman_sub = ghuman.subgraph(list(ghuman.nodes())[100:200])
gmouse_sub = gmouse.subgraph(list(gmouse.nodes())[100:200])
human_nodelist = list(ghuman_sub.nodes())
mouse_nodelist = list(gmouse_sub.nodes())
human_map = {k:i for i, k in enumerate(human_nodelist)}
mouse_map = {k:i for i, k in enumerate(mouse_nodelist)}

In [57]:
gmouse_sub

## Network alignment

### Isorank

In [61]:
from gmundo.alignment import isorank
it = 10
ch = 1
matches = isorank(ghuman_sub, gmouse_sub, human_map, mouse_map, 1, 50, iterations = it, saveto=f"isomap_mappings_{it}_{ch}.tsv",
                 rowname="human",
                 colname="mouse")

Running iterations 0...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 7118.17it/s]


Running iterations 1...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 7034.18it/s]


Running iterations 2...


...

Running iterations 8...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 7052.76it/s]


Running iterations 9...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:01<00:00, 6982.32it/s]

Number of pairings... 50
Len(ps) = 10000 Len(qs) = 10000
Break complete
50


In [62]:
df_a = pd.read_csv("isomap_mappings_2_1.tsv", sep = "\t")
df_b = pd.read_csv("isomap_mappings_10_1.tsv", sep = "\t")

df_a_b = pd.merge(df_a, df_b, how="outer", on=["human", "mouse"], indicator=True).query('_merge == "both"').drop("_merge", axis = 1)
df_a_b

,human,mouse,weight_x,weight_y
0,13555,2252,0.233165,0.300422
1,233908,3984,0.174874,0.200379
2,12421,2260,0.174874,0.200379
3,21687,4301,0.136013,0.035925
5,22030,11337,0.129536,0.113277
6,12125,904,0.125550,0.019705
10,21349,2224,0.116582,0.100336
13,20466,3084,0.116582,0.100336
22,13368,3791,0.077722,0.112656
25,53331,149428,0.074946,0.029947


In [37]:
df_5 = pd.read_csv("isomap_mappings_5.tsv", sep = "\t")
df_10 = pd.read_csv("isomap_mappings_10.tsv", sep = "\t")

df_5_10 = pd.merge(df_5, df_10, how="outer", on=["human", "mouse"], indicator=True).query('_merge == "both"').drop("_merge", axis = 1)
df_5_10

,human,mouse,weight_x,weight_y


In [24]:
# this isorank code takes too much space, so have to optimize this properly
def randomized_mapping(n1, n2, n_mapping):
    rn1 = np.random.permutation(len(n1))[:n_mapping]
    rn2 = np.random.permutation(len(n2))[:n_mapping]
    i_mapping = list(zip(rn1, rn2))
    mapping   = [(n1[p], n2[q]) for p, q in zip(rn1, rn2)]
    return mapping, i_mapping
mapping, i_mapping = randomized_mapping(human_nodelist, mouse_nodelist, 50)

### Hubalign

In [ ]:
from gmundo.alignment_utils import seq_sim_file_from_refseq_to_entrezgene

# Here we convert sequence similarity file from refseq format to entrezgene if needed.
# After that the resulting file can be used to improve hubalign network alignment.
seq_sim_file_from_refseq_to_entrezgene("../data/sequence_similarity/human-mouse/human-mouse-seq-sim-refseq.tsv", "../data/sequence_similarity/human-mouse/human-mouse-seq-sim-entrezgene.tsv", 'MOUSE', 'HUMAN')

In [9]:
from gmundo.alignment_utils import read_mapping_from_hubalign_alignment_file

# Read mapping of the nodes of 2 different networks from Hubalign network alignment result file. 
# Alignment itself is performed out of scope of this notebook.
mapping, i_mapping = read_mapping_from_hubalign_alignment_file("../data/hubalign_files/mouse-human-alignment-seq-sim-0.7", human_nodelist, mouse_nodelist, 150, is_need_to_swap_columns = True)

## Coembedding

In [10]:
# Computing the DSD matrices for human and mouse
# This function takes a lot of time, so I have saved the numpy file to save time
from gmundo.linalg import compute_dsd_embedding
mouse_dsd  = compute_dsd_embedding(gmouse, mouse_nodelist)
human_dsd  = compute_dsd_embedding(ghuman, human_nodelist)

In [11]:
import numpy as np
np.save("human_dsd_emb.npy", human_dsd)
np.save("mouse_dsd_emb.npy", mouse_dsd)
#human_dsd = np.load("human_dsd_emb.npy")
#mouse_dsd = np.load("mouse_dsd_emb.npy")

In [ ]:
from scipy.spatial.distance import squareform, pdist

human_dist = squareform(pdist(human_dsd))
mouse_dist = squareform(pdist(mouse_dsd))

In [ ]:
np.save("human_dist.npy", human_dist)
np.save("mouse_dist.npy", mouse_dist)
#human_dist = np.load("human_dist.npy")
#mouse_dist = np.load("mouse_dist.npy")

In [ ]:
from sklearn.metrics.pairwise import laplacian_kernel
gamma = 1 / 10
human_rbf = laplacian_kernel(human_dist)
mouse_rbf = laplacian_kernel(mouse_dist)
np.save("human_rbf_0.1.npy", human_rbf)
np.save("mouse_rbf_0.1.npy", mouse_rbf)

In [ ]:
from gmundo.coembed import coembed_networks
human_rbf = np.load("human_rbf_0.1.npy")
mouse_rbf = np.load("mouse_rbf_0.1.npy")
# in this case mouse is a target and human is a source organism
munk = coembed_networks(human_rbf, mouse_rbf, i_mapping, verbose = True)
np.save("munk.npy", munk)

### PREDICTION using MUNK associations computed above

In [23]:
from gmundo.prediction.go_process import get_go_labels

filter_label = {"namespace": "molecular_function", "min_level": 5}
filter_prot  = {"namespace": "molecular_function", "lower_bound": 50}
human_labels, human_go_prots_dict = get_go_labels(filter_prot, filter_label, human_nodelist, "../data/go_files/gene2go", "../data/go_files/go-basic.obo", 9606, verbose = True)

HMS:0:00:06.192351 335,350 annotations, 20,702 genes, 18,726 GOs, 1 taxids READ: ../data/go_files/gene2go 
18674 IDs in loaded association branch, molecular_function
  EXISTS: ../data/go_files/go-basic.obo
../data/go_files/go-basic.obo: fmt(1.2) rel(2021-12-15) 47,157 Terms; optional_attrs(relationship)
Number of GO-terms: 32


In [24]:
filter_label = {"namespace": "molecular_function", "min_level": 5}
filter_prot  = {"namespace": "molecular_function", "lower_bound": 20}
mouse_labels, mouse_go_prots_dict = get_go_labels(filter_prot, filter_label, mouse_nodelist, "../data/go_files/gene2go", "../data/go_files/go-basic.obo", 10090, verbose = True)

HMS:0:00:06.016468 419,936 annotations, 29,777 genes, 18,906 GOs, 1 taxids READ: ../data/go_files/gene2go 
18880 IDs in loaded association branch, molecular_function
  EXISTS: ../data/go_files/go-basic.obo
../data/go_files/go-basic.obo: fmt(1.2) rel(2021-12-15) 47,157 Terms; optional_attrs(relationship)
Number of GO-terms: 39


In [25]:
with open("human.json", "r") as hj:
    h_entrez_id = json.load(hj)
with open("mouse.json", "r") as mj:
    m_entrez_id = json.load(mj)
def get_prot_go_dict(go_prot_dict, entrez_id_map):
    prot_go = {}
    for l in go_prot_dict:
        for p in go_prot_dict[l]:
            if entrez_id_map[str(p)] not in prot_go:
                prot_go[entrez_id_map[str(p)]] = [l]
            else:
                prot_go[entrez_id_map[str(p)]].append(l)
    return prot_go
mouse_prot_go = get_prot_go_dict(mouse_go_prots_dict, m_entrez_id)
human_prot_go = get_prot_go_dict(human_go_prots_dict, h_entrez_id)


In [26]:
# Construct the target and MUNK maps
mouse_rbf = np.load("mouse_rbf_0.1.npy")
mouse_neighbors = np.argsort(-mouse_rbf, axis = 1)
# Need to discuss this
mouse_neighbors = mouse_neighbors[:, :20]

In [27]:
munk  = np.load("munk.npy")
munk_neighbors = np.argsort(-munk, axis = 1)
munk_neighbors = munk_neighbors[:, :20]

def convert_to_dict(npy_neighbors):
    ndict = {}
    n, _  = npy_neighbors.shape
    for i in range(n):
        ndict[i] = npy_neighbors[i, :]
    return ndict

munk_neigh_dict = convert_to_dict(munk_neighbors)
mouse_neigh_dict = convert_to_dict(mouse_neighbors)

In [67]:
# Constructing a predictor function
from gmundo.prediction.predict import mundo_predict
def construct_predictor_mundo(target_neighbors, munk_neighbors, source_prot_go, n_neighbors = 20, MUNK_weight = 0.25):
    def predictor(target_prot_go):
        return mundo_predict(target_neighbors,
                             munk_neighbors,
                             n_neighbors,
                             target_prot_go,
                             source_prot_go,
                             MUNK_weight)
    return predictor

In [68]:
from gmundo.prediction.scoring import kfoldcv
accs = kfoldcv(5,
              mouse_prot_go,
              construct_predictor_mundo(mouse_neigh_dict,
                                        munk_neigh_dict,
                                       human_prot_go)
              )

In [69]:
np.mean(accs)

0.26574690516296356